# Домашнее задание по теме  
«Регрессия Decision Tree»

Использую набор ранее обработанных данных из прошлых заданий.  Столбец “Premium Amount” является целевой переменной

## Импорт библиотек, загрузка данных

In [1]:
# Импорт библиотек
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
from sklearn.preprocessing import (MinMaxScaler, StandardScaler, RobustScaler, 
                                   MaxAbsScaler, LabelEncoder, OneHotEncoder
)       
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, PowerTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor

In [2]:
# Загрузка файла с данными из архива
archive_path = 'input_data/train.zip'
with zipfile.ZipFile(archive_path, 'r') as archive:
 
    with archive.open('train.csv') as file:
        df = pd.read_csv(file)

In [3]:
df.describe(include='all')

,age,annual_income,marital_status,number_of_dependents,education_level,occupation,health_score,policy_type,previous_claims,vehicle_age,credit_score,insurance_duration,exercise_frequency,property_type,premium_amount,gender_Male,smoking_status_Yes,location_Suburban,location_Urban
count,1.050007e+06,1.050007e+06,1.050007e+06,1.050007e+06,1.050007e+06,1.050007e+06,1.050007e+06,1.050007e+06,1.050007e+06,1.050007e+06,1.050007e+06,1.050007e+06,1.050007e+06,1.050007e+06,1.050007e+06,1.050007e+06,1.050007e+06,1.050007e+06,1.050007e+06
mean,4.114466e+01,3.127431e+04,1.003080e+00,1.826691e+00,1.506036e+00,6.955592e-01,2.557233e+01,1.002804e+00,9.619050e-01,9.565279e+00,5.941532e+02,5.017622e+00,1.515659e+00,1.000079e+00,1.081938e+03,5.021224e-01,5.012043e-01,3.350435e-01,3.308597e-01
std,1.354263e+01,2.800532e+04,8.163006e-01,1.469989e+00,1.123290e+00,8.197235e-01,1.216933e+01,8.165291e-01,7.128295e-01,5.775979e+00,1.407340e+02,2.594273e+00,1.117939e+00,8.167502e-01,8.000198e+02,4.999957e-01,4.999988e-01,4.720059e-01,4.705229e-01
min,1.800000e+01,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.012237e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e+02,1.000000e+00,0.000000e+00,0.000000e+00,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.000000e+01,8.404000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.591187e+01,0.000000e+00,0.000000e+00,5.000000e+00,4.850000e+02,3.000000e+00,1.000000e+00,0.000000e+00,5.150000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,4.100000e+01,2.399100e+04,1.000000e+00,2.000000e+00,2.000000e+00,0.000000e+00,2.455183e+01,1.000000e+00,1.000000e+00,1.000000e+01,5.960000e+02,5.000000e+00,2.000000e+00,1.000000e+00,8.740000e+02,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
75%,5.300000e+01,4.425050e+04,2.000000e+00,3.000000e+00,3.000000e+00,1.000000e+00,3.445247e+01,2.000000e+00,1.000000e+00,1.500000e+01,7.060000e+02,7.000000e+00,3.000000e+00,2.000000e+00,1.513000e+03,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
max,6.400000e+01,9.802025e+04,2.000000e+00,4.000000e+00,3.000000e+00,2.000000e+00,5.897591e+01,2.000000e+00,2.500000e+00,1.900000e+01,8.490000e+02,9.000000e+00,3.000000e+00,2.000000e+00,3.010000e+03,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


## Проведение регрессии методом решающих деревьев

### Выбор лудших признаков

In [4]:
# Выбор лучших признаков
X = df.drop(columns=['premium_amount'])  # Все признаки
y = df['premium_amount']  # Целевая переменная

k = 6  # Количество признаков в отборе
selector = SelectKBest(score_func=f_regression, k=k)

X_new = selector.fit_transform(X, y)

# Индексы выбранных признаков
selected_indices = selector.get_support(indices=True)

# Названия выбранных признаков
selected_features = X.columns[selected_indices]

print("Выбранные признаки:")
print(selected_features)

Выбранные признаки:
Index(['age', 'annual_income', 'number_of_dependents', 'health_score',
       'previous_claims', 'credit_score'],
      dtype='object')


In [5]:
df_cut = df[['age', 'annual_income', 'number_of_dependents', 'health_score',
       'previous_claims', 'credit_score', 'premium_amount']]

In [6]:
# Разделение данных на признаки (X) и целевую переменную (y)
X = df_cut.drop(columns=['premium_amount'])
y = df_cut['premium_amount']  # Целевая переменная
# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
param_grid = {
    'max_depth': [None, 3, 5, 10, 15],  # Глубина дерева
    'min_samples_split': [2, 5, 10],     # Минимальное количество образцов для разделения узла
    'min_samples_leaf': [1, 2, 4],       # Минимальное количество образцов в листе
    'max_features': ['auto', 'sqrt', 'log2'],  # Количество признаков для поиска лучшего разделения
    'random_state': [42]  # Для воспроизводимости
}

In [ ]:
# Модель DecisionTreeRegressor
model = DecisionTreeRegressor()

# Создаем объект GridSearchCV
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='r2',  # Метрика для оптимизации 
    cv=5,  # Количество фолдов для кросс-валидации
    n_jobs=-1  # Использовать все доступные ядра процессора
)

# Обучение модели с поиском по сетке гиперпараметров
grid_search.fit(X_train, y_train)
# Лучшие параметры
best_params = grid_search.best_params_

In [9]:
print("Лучшие параметры:", best_params)
# best_model = grid_search.best_estimator_

Лучшие параметры: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 10, 'random_state': 42}


### Создание модели регрессии Decision Tree

In [10]:
rdt_model = DecisionTreeRegressor(**best_params)

rdt_model.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=10, max_features='sqrt', min_samples_split=10,
                      random_state=42)

In [11]:
y_pred_rdt = rdt_model.predict(X_test)

mse_rdt = mean_squared_error(y_test, y_pred_rdt)
r2_rdt = r2_score(y_test, y_pred_rdt)
mae_rdt = mean_absolute_error(y_test, y_pred_rdt)
print(f"Regression DT - MSE: {mse_rdt}, R2 Score: {r2_rdt}, MAE: {mae_rdt}")

Regression DT - MSE: 621365.424446415, R2 Score: 0.030203941119623323, MAE: 623.7726711689362


### Сравнение с моделью KNN

In [12]:
# Масштабирование с помощью MinMaxScaler
minmax_scaler = MinMaxScaler()
scaled_data_minmax = minmax_scaler.fit_transform(df_cut)
df_reg = pd.DataFrame(scaled_data_minmax, columns=df_cut.columns)

In [13]:
# Разделение данных на признаки (X) и целевую переменную (y)
X_r = df_reg.drop(columns=['premium_amount'])
y_r = df_reg['premium_amount']  # Целевая переменная
# Разделение данных на обучающую и тестовую выборки
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
knn_model = KNeighborsRegressor(n_neighbors=150) 
knn_model.fit(X_train_r, y_train_r)

y_pred_knn = knn_model.predict(X_test_r)

mse_knn = mean_squared_error(y_test_r, y_pred_knn)
r2_knn = r2_score(y_test_r, y_pred_knn)
mae_knn = mean_absolute_error(y_test_r, y_pred_knn)
print(f"KNN - MSE: {mse_knn}, R2 Score: {r2_knn}, MAE: {mae_knn}")

KNN - MSE: 633775.4488561219, R2 Score: 0.010834996067900726, MAE: 635.7329181626842


### Общие выводы:
- Regression DT - MSE: 621365.424446415, R2 Score: 0.030203941119623323, MAE: 623.7726711689362
- KNN -           MSE: 633775.448856121, R2 Score: 0.010834996067900726, MAE: 635.7329181626842

- Модель DecisionTreeRegressor на этих данных показала несколько лучшие результаты, но по прежнему делает значительные ошибки в предсказании. 
- Модель объясняет 3% дисперсии целевой переменной.
- Текущий набор данных показывает плохие результаты на всех изученных регрессиях, но DecisionTreeRegressor немногим лучше